# 音声・字幕からAnkiカード生成ツール

このノートブックは、動画や音声ファイルとSRT字幕を使って、Anki学習カードを自動生成するためのツールです。

## 主な機能
- 動画から音声を抽出
- 音声ファイルを任意の時間で分割
- SRT字幕のタイムスタンプに基づいて音声を文ごとに分割
- 音声セグメントと字幕テキストを組み合わせたAnki用CSVファイルの生成

## 使用シーン
字幕付きの動画や音声教材から、語学学習用のAnkiフラッシュカードを自動生成する際に使用します。


In [ ]:
#動画から音声を抽出する
from moviepy import VideoFileClip

video_path = "/path/to/your/video.mp4"      # 動画ファイルのパス
audio_path = "/path/to/output/audio.mp3"     # 出力音声ファイル名

clip = VideoFileClip(video_path)
clip.audio.write_audiofile(audio_path)

clip.close()


In [ ]:
#時間に応じて自由に音声を切り取る
from moviepy import AudioFileClip

# 音声ファイルを読み込む
audio = AudioFileClip("/path/to/your/audio.mp3")

# 切り取りたい時間帯を設定（形式：MM:SS または HH:MM:SS）
segments = [
    ("05:00", "10:00"),
    #("12:16", "25:14"),
    # さらに追加可能
]

# 音声を分割
for i, (start, end) in enumerate(segments, start=1):
    clip = audio.subclipped(start, end)
    
    start_clean = start.replace(":","_")
    end_clean = end.replace(":","_")
    filename = f"{start_clean}-{end_clean}.mp3"
    
    clip.write_audiofile(filename, codec="libmp3lame", logger=None)
    print(f"保存完了: {filename}")

audio.close()


In [ ]:
#srtファイルのタイムラインに基づいて、各文に対応する音声を分割する
import os
import pysrt
from moviepy import AudioFileClip

# ファイル名を設定
audio_file = "/path/to/your/audio.mp3"
srt_file = "/path/to/your/subtitle.srt"
output_folder = "/path/to/output/segments"

# 出力フォルダを作成
os.makedirs(output_folder, exist_ok=True)

# 音声ファイルを読み込む
audio = AudioFileClip(audio_file)

# 字幕ファイルを読み込む
subs = pysrt.open(srt_file)

# 音声を分割
for sub in subs:
    index = sub.index
    start = sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds + sub.start.milliseconds / 1000
    end = sub.end.hours * 3600 + sub.end.minutes * 60 + sub.end.seconds + sub.end.milliseconds / 1000

    clip = audio.subclipped(start, end)
    output_path = os.path.join(output_folder, f"segment_{index:04}.mp3")
    clip.write_audiofile(output_path, codec='libmp3lame', logger=None)

    print(f"保存完了: {output_path}")

audio.close()


In [ ]:
#csvファイルを生成し、音声と字幕をマッチングする
#ankiにインポート、音声はC:\Users\username\AppData\Roaming\Anki2\ユーザー名\collection.mediaに配置
import pysrt
import csv
import os

srt_file = "/path/to/your/subtitle.srt"
audio_folder = "/path/to/audio/segments"
output_csv = "/path/to/output/anki_cards.csv"

subs = pysrt.open(srt_file, encoding='utf-8')

rows = []

for sub in subs:
    n = sub.index  # 字幕番号：1, 2, ..., 999
    audio_file = f"segment_{n:04}.mp3"  # ゼロ埋めで4桁に

    # 音声ファイルが存在するか確認
    audio_path = os.path.join(audio_folder, audio_file)
    if not os.path.exists(audio_path):
        print(f"⚠️ 音声が見つかりません: {audio_file}、スキップします")
        continue

    # 字幕テキストを取得し、改行を削除
    text = sub.text.replace('\n', ' ').strip()

    # 1行を追加：音声フィールド + テキストフィールド
    rows.append([f"[sound:{audio_file}]", text])

# CSV ファイルに書き込む
with open(output_csv, "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerows(rows)

print(f"✅ CSV を生成しました: {output_csv}（合計 {len(rows)} 件）")


## Ankiへのインポート方法

### 1. 音声ファイルの配置
生成された音声セグメントファイル（`segment_0001.mp3`など）を、Ankiのメディアフォルダにコピーします。

**Ankiメディアフォルダの場所：**
```
C:\Users\ユーザー名\AppData\Roaming\Anki2\プロファイル名\collection.media
```

### 2. CSVファイルのインポート
1. Ankiを起動し、対象のデッキを選択
2. メニューから「ファイル」→「読み込み」を選択
3. 生成されたCSVファイル（例：`anki_cards.csv`）を選択
4. フィールドの区切り文字が「カンマ」になっていることを確認
5. 「インポート」をクリック

### 3. ノートタイプの設定
- 1列目：音声フィールド（`[sound:ファイル名.mp3]`形式）
- 2列目：テキストフィールド（字幕の内容）

インポートが完了すると、音声付きの学習カードが自動的に作成され、すぐに学習を開始できます。

### 注意事項
- 音声ファイルは必ずCSVインポート前にメディアフォルダに配置してください
- ファイル名が正しく対応していることを確認してください
- 既存のノートタイプを使用する場合は、フィールドの順序を合わせてください
